In [1]:
import pandas as pd
from utils import *

2022-12-08 14:52:40.340828: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-08 14:52:40.482146: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-08 14:52:41.255917: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-08 14:52:41.255988: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
ans_df = pd.read_csv("/nfs/nas-6.1/wclu/AICUP/full_valid.csv")
max_q = 0
max_r = 0

In [3]:
path = "/nfs/nas-6.1/wclu/corrected_models/janeel/muppet-roberta-base-finetuned-squad-62+2-b_8-seed_26"
for step in range(200, 3001, 200): 
    try:
        pred_df = pd.read_csv(f"{path}/checkpoint-{step}.csv", names=['id', "q'", "r'"]).fillna("")
        q = pred_df[['id', "q'"]]
        empty_r = pd.DataFrame(data={"r'": ["" for _ in range(1598)]})
        r = pred_df[['id', "r'"]]
        empty_q = pd.DataFrame(data={"q'": ["" for _ in range(1598)]})

        q_frame=pd.concat([q, empty_r], axis=1).reset_index(drop=True)
        r_frame=pd.concat([r, empty_q], axis=1).reset_index(drop=True)

        lcs = compute_lcs_scores(q_frame, ans_df)
        score_q = compute_final_score(lcs)

        lcs = compute_lcs_scores(r_frame, ans_df)
        score_r = compute_final_score(lcs)
        if score_q >= max_q:
            best_q = q
            step_q = step
            q_path = path
            max_q = score_q
        if score_r >= max_r:
            best_r = r
            step_r = step
            r_path = path
            max_r = score_r
        print(f"ckpt{step} q_score: {score_q} r_score: {score_r}")
    except:
        continue

best_pred = pd.concat([best_q, best_r["r'"]], axis=1).reset_index(drop=True)
lcs = compute_lcs_scores(best_pred, ans_df)
score = compute_final_score(lcs)
print(f"model for q: {q_path}_{step_q}, score: {max_q}")
print(f"model for r: {r_path}_{step_r}, score: {max_r}")
print(score)

    

ckpt400 q_score: 0.40763342719677015 r_score: 0.42461873316277177
ckpt600 q_score: 0.4132923237647221 r_score: 0.4255104838380309
ckpt800 q_score: 0.41462833369885077 r_score: 0.4251512192573634
ckpt1000 q_score: 0.42055276484403126 r_score: 0.42567992270945093
ckpt1200 q_score: 0.4211159942476636 r_score: 0.4282665144641968
ckpt1400 q_score: 0.41730438881022314 r_score: 0.42667509945358123
ckpt1600 q_score: 0.4192010009340434 r_score: 0.4224954835939029
ckpt1800 q_score: 0.4198293640004861 r_score: 0.4278395035039769
ckpt2000 q_score: 0.416476962538265 r_score: 0.4288107756497183
ckpt2200 q_score: 0.41661275703542766 r_score: 0.4252958940100563
ckpt2400 q_score: 0.41923210908391034 r_score: 0.4314555594479848
ckpt2600 q_score: 0.41798829787509156 r_score: 0.4277397935961429
ckpt2800 q_score: 0.41267919794575614 r_score: 0.42949547127586635
ckpt3000 q_score: 0.41392411742168206 r_score: 0.4302834130138141
model for q: /nfs/nas-6.1/wclu/corrected_models/janeel/muppet-roberta-base-finetu